# Federated Iris Recognition - Client1
This notebook acts as **Client1** in the federated learning experiment using Flower.


In [ ]:
!pip install -q flwr tensorflow

In [1]:
import flwr as fl
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from model import build_model

# ==== Data configuration ====
TRAIN_DIR = r"C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset\Clients\client1\train"
VAL_DIR   = r"C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset\Clients\client1\val"
TEST_DIR  = r"C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset\Clients\client1\test"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# No validation split needed
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

# Load from manually split folders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


# Build model with correct number of output classes
NUM_CLASSES = train_generator.num_classes
model = build_model(NUM_CLASSES)




Found 148 images belonging to 74 classes.
Found 444 images belonging to 74 classes.
Found 148 images belonging to 74 classes.


In [3]:
import flwr as fl
from model import build_model
# ==== Flower client definition ====
class IrisClient(fl.client.NumPyClient):
    def __init__(self, model, train_gen, val_gen, test_gen):
        self.model = model
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.test_gen = test_gen

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(
            self.train_gen,
            validation_data=self.val_gen,
            epochs=1,
            steps_per_epoch=len(self.train_gen),
            verbose=1
        )
        return self.model.get_weights(), len(self.train_gen), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_gen, verbose=0)
        return loss, len(self.test_gen), {"accuracy": float(accuracy)}

# ==== Start Flower client ====
fl.client.start_client(
    server_address="127.0.0.1:8081",
    client=IrisClient(model, train_generator, val_generator, test_gen).to_client()
)



	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message 6afcd53c-e9fc-4c6b-99f5-792392b66976
INFO :      Sent reply
INFO :      
INFO :      Received: train message 7ceee6ee-20cd-4d88-97b6-2eb01bb7059b


14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.3051 - loss: 3.2464 - val_accuracy: 0.5608 - val_loss: 2.6018


INFO :      Sent reply
INFO :      
INFO :      Received: train message 1b5c39b3-0be9-4afd-8eee-c8403673541c


14/14 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.5531 - loss: 2.7825 - val_accuracy: 0.6959 - val_loss: 2.3146


INFO :      Sent reply
INFO :      
INFO :      Received: train message d44ad17f-2f7d-4a18-8652-886dc1de6694


...

INFO :      Sent reply
INFO :      
INFO :      Received: train message 5d74e6fe-c056-45ec-acc8-0295bf0e7ab1


14/14 ━━━━━━━━━━━━━━━━━━━━ 50s 4s/step - accuracy: 0.9863 - loss: 0.4305 - val_accuracy: 0.9459 - val_loss: 0.6439


INFO :      Sent reply
INFO :      
INFO :      Received: train message 9b88914d-43c1-4276-99d0-1fa2414c6215


14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.9867 - loss: 0.3944 - val_accuracy: 0.9662 - val_loss: 0.6195


INFO :      Sent reply
INFO :      
INFO :      Received: reconnect message 208ab7f2-b5a2-4831-b0b8-465ae2eb9ebe
INFO :      Disconnect and shut down


In [4]:
test_loss, test_acc = model.evaluate(test_gen, verbose=2)
print("\nTest accuracy:", test_acc)

5/5 - 4s - 879ms/step - accuracy: 0.9527 - loss: 0.5665

Test accuracy: 0.9527027010917664
